# Daten kombinieren

**Inhalt:** Verschiedene Quellen kombinieren, optisch und tabellarisch auswerten

**Nötige Skills:** Daten explorieren, filtern, klassifizieren

**Lernziele:**
- Besser verstehen, wie Merge funktioniert
- Daten kombinieren, Kategorien bilden und auswerten
- Daten exportieren und in anderem Programm visualisieren

# Das Beispiel

Nationalratswahlen. Wir analysieren die Veränderung über die letzten 20 Jahre:
1. über alle Gemeinden hinweg
2. für bestimmte Gruppen von Gemeinden (zB je nach Sprachregion)
3. zum Schluss stellen wir die Ergebnisse in Karten dar

Angelehnt an: https://www.republik.ch/2019/10/21/gruener-jurabogen-und-gruenliberales-zuerich

**Quellen:**
- Wahlergebnisse beim BFS: Daten gibts beim BFS: https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-1702020000_105/px-x-1702020000_105/px-x-1702020000_105.px
- Diverse Daten zu Gemeinden, zusammengetragen beim BFS



## Vorbereitung

Wir importieren ausnahmsweise etwas mehr Bibliotheken als sonst...

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
import geopandas as gpd

## Daten laden

**Wahlergebnisse**

Wir haben uns schonmal ein hübsches geputztes File vorbereitet.

Das ist unser Hauptfile - es enthält die Wähleranteile der Parteien in den Jahren 1999 und 2019.

In [ ]:
path = 'dataprojects/wahlen/Wahlergebnisse 1999 und 2019 in Gemeinden.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
df

**Gemeindetypologie**

Wir haben eine Hilfsdatei - sie enthält für jede Gemeinde ein paar Zusatzinformationen:
- Welchem Raumtyp sie zugehört
- In welcher Grossregion sie liegt
- Welche Sprache gesprochen wird
- Der Ausländeranteil
- Wie stark die Bevölkerugungszahl gewachsen ist

In [ ]:
df_typen = pd.read_excel('dataprojects/wahlen/Gemeindetypologie.xlsx')

In [ ]:
df_typen

Unser Ziel ist, die Informationen aus dieser Tabelle mit den Wähleranteilen zu verbinden, so dass wir gruppierte Auswertungen machen können.

## Vorbereitung

Zuerst müssen wir uns die Daten nochmals ganz genau anschauen.

In [ ]:
df.head(20)

**NA-Values**

Wir müssen uns überlegen: Was heissen leere Zeilen (NaN) genau?

- Wir könnten sagen: Diese Einträge wollen wir gar nicht anschauen...
- ... oder wir könnten sagen: Wähleranteil der betreffenden Partei = 0

Pandas bietet eine praktische Funktion, um NaN zu ersetzen: `fillna()`

In [ ]:
df['2019'] = df['2019'].fillna(0)
df['1999'] = df['1999'].fillna(0)

In [ ]:
df.head(20)

**Veränderung**

Als nächstes überlegen wir uns, was uns eigentlich interessiert.

Wir kennen bereits für jede Partei und Gemeinde:
- den Wähleranteil 1999
- den Wähleranteil 2019

Was wir noch berechnen müssen:
- die Veränderung von 1999 bis 2019

Hier ist ein guter Ort, um das zu tun.

In [ ]:
df['Differenz'] = df['2019'] - df['1999']
df

**Parteien**

Okay... welche Parteien wollen wir wirklich anschauen?

Es hat ziemlich viele:

In [ ]:
df['Partei_Name'].unique()

Lasst uns für den weiteren Verlauf der Analyse eine Auswahl treffen:

In [ ]:
parties = ['FDP', 'CVP', 'SP', 'SVP', 'GPS', 'GLP', 'BDP']

In [ ]:
df = df[df['Partei_Name'].isin(parties)]

In [ ]:
df

## 1. Veränderung über alle Gemeinden hinweg

Wir starten mit ein paar Aufwärmübungen. Das kennen wir bereits:

**Quiz:** Wie waren die Wähleranteile je nach Partei im Jahr 1999 im Schnitt?

Lass uns das plotten:

Und wie hoch waren die Wähleranteile der Parteien 2019 im Schnitt?

Machen wir nochmals einen Plot...

Wie viele Wählerprozente haben die Parteien im Schnitt zugelegt?

Plot:

**Schöne Farben**

Übrigens... wir können für unsere Plots noch schönere Parteifarben hinzufügen.

Dazu definieren wir uns einen Dictionary von Parteien und dazugehörigen Farben:

In [ ]:
colors = {
  "SVP": "#4B8A3E",
  "FDP": "#3872B5",
  "CVP": "#D6862B",
  "BDP": "#E6C820",
  "GLP": "#C4C43D",
  "SP": "#F0554D",
  "GPS": "#84B547"
}

So können wir zum Beispiel nach der Farbe der SVP fragen:

In [ ]:
colors['SVP']

Pandas ist leider zu blöd, um diesen Dictionary einfach als Parameter für die Plotfunktion zu verstehen.

Daher müssen wir einen kleinen Trick anwenden.

Die Tabelle, die wir plotten wollen, speichern wir dazu kurz in einem Dataframe:

In [ ]:
df_temp = df.groupby('Partei_Name')['Differenz'].mean().sort_values()
df_temp

Für den Plot brauchen wir eine Liste von Farben, die entsprechend dieser Reihenfolge geordnet sind.

Wir machen das, in dem wir "list comprehension" verwenden. Das heisst: wie nehmen die Indexwerte der Tabelle (also die Parteien) und schlagen für jeden Eintrag die passende Farbet nach.

In [ ]:
[colors[key] for key in df_temp.index]

In [ ]:
df_temp.plot(kind='barh', color=[colors[key] for key in df_temp.index])

## 2. Veränderung über Gruppen von Gemeinden hinweg

Nun kommen wir endlich zu dem Teil, der uns eigentlich interessiert: gruppierte Auswertungen!

Dazu müssen wir uns entscheiden:
- welche Partei wollen wir analysieren?
- welche Gemeindegruppierung interessiert uns?
- welchen Wählerwert wollen wir anschauen?

**Frage:** Was könnte eine mögliche Fragestellung sein?

In [ ]:
# Hier Platz für Antwort



### a) anhand von qualitativen Merkmalen

Wir beginnen mal mit der Raumgliederung.

Und nehmen eine Partei in den Fokus...

In [ ]:
partei = 'SVP'

... und filtern unseren Datensatz nach dieser Partei:

In [ ]:
df_partei = df[df['Partei_Name'] == partei]

In [ ]:
df_partei.head()

Jetzt folgt der entscheidende Schritt: die Gruppierung. Wir müssen zu jeder Gemeinde die Info hinzufügen, welchem Raumtyp sie angehört. (Remember, diese Infos sind im Hilfsdataframe hinterlegt, das wir geladen haben:

In [ ]:
df_typen.head()

Um die zwei Tabellen zu "verbinden", können wir die Funktion `merge()` benutzen:
- Methode ("how"): nur die Gemeinden, die in beiden Tabellen vorkommen
- Datenfeld, das fürs matching verwendet wird ("on"): Gemeinde_ID

In [ ]:
df_partei2 = df_partei.merge(df_typen, how='inner', on='Gemeinde_ID')

In [ ]:
df_partei2.head()

Jetzt haben wir ein Dataframe, das wir ganz einfach mit `groupby()` auswerten können.

In [ ]:
df_partei2.groupby('Raumtyp')['Differenz'].mean()

Für unseren Plot können wir die Farb-Zuordnung von vorher wieder benutzen! :-)

In [ ]:
df_partei2.groupby('Raumtyp')['Differenz'].mean().plot(
    kind='barh', 
    color=colors[partei],
    title=('Veränderung je nach Gemeinde-Raumtyp, ' + partei))

### Übungsfragen

**Quiz:** Wie hoch waren 1999 die Wähleranteile der SP je nach Grossregion? Und 2019?

Lösungsweg:
- Datensatz nach der Partei filtern
- Gemeinde-Informationen hinzufügen
- Datensatz nach Gemeindegruppe auswerten

Erstellen Sie einen Plot und beschriften Sie ihn.

**Quiz:** Wie haben sich die Wähleranteile der CVP je nach Sprachregion verändert?

**Quiz:** Wie hat sich der Wähleranteil der Grünen je nach Ausländerantei einer Gemeinde verändert?

### b) anhand von quantitativen Merkmalen

Der Ausländeranteil ist eine kontinuierliche Grössenangabe - es gibt hier noch keine Kategorien!

In [ ]:
partei = 'GPS'

In [ ]:
df_partei = df[df['Partei_Name'] == partei]

In [ ]:
df_partei2 = df_partei.merge(df_typen, how='inner', on='Gemeinde_ID')

In [ ]:
df_partei2.plot(kind='scatter', x='Ausländeranteil', y='Differenz', color=colors[partei])

Wir müssen zuerst unsere Kategorien basteln!

Dazu müssen wir zuerst mal wissen:
- anhand von welchen Grenzen wollen wir die Kategorien bilden? (z.B. 0-10%, 10-15%, etc.)
- wie wollen wir diese Kategorien beschriften?

Wir schauen erstmal, wie die Werte ungefähr verteilt sind.

In [ ]:
df_partei2['Ausländeranteil'].hist(bins=50)

Als nächstes erstellen wir zwei Listen:

In [ ]:
bin_values = [
    0,
    10,
    15,
    20,
    25,
    np.inf
]

In [ ]:
bin_codes = [
    "10 Prozent oder weniger",
    "10 bis 15 Prozent",
    "15 bis 20 Prozent",
    "20 bis 25 Prozent",
    "25 Prozent oder mehr"
]

Es gibt in Pandas eine praktische Funktion, mit der man die Kategorisierung vornehmen kann: `pd.cut()`

In [ ]:
pd.cut(df_partei2['Ausländeranteil'], bin_values, labels=bin_codes, include_lowest=True)

Jetzt speichern wir diese Kategorien einfach in einer neuen Spalte:

In [ ]:
df_partei2['Ausländeranteil_Kat'] = pd.cut(df_partei2['Ausländeranteil'], bin_values, labels=bin_codes, include_lowest=True).astype(str)

In [ ]:
df_partei2

Nun können wir unsere Auswertung wie zuvor vornehmen:

In [ ]:
df_partei2.groupby('Ausländeranteil_Kat')['Differenz'].mean()

In [ ]:
df_partei2.groupby('Ausländeranteil_Kat')['Differenz'].mean().plot(
    kind='barh',
    color=colors[partei],
    title=('Veränderung je nach Ausländeranteil, ' + partei))

### Übungsfragen

**Quiz:** Wie hat sich der Wähleranteil der CVP in schnell, wie in langsam wachsenden Gemeinden entwickelt?

Lösungsweg:
- Datensatz nach der Partei filtern
- Kategorisierung fürs Bevölkerungswachstum erstellen
- Gemeinde-Informationen hinzufügen
- Datensatz nach Gemeindegruppe auswerten

Erstellen Sie einen Plot und beschriften Sie ihn.

## 3. Räumliche Darstellung der Auswertungen

Es wäre nett, wenn wir die Veränderungen nicht nur als Balkenchart, sondern auch gleich auf einer Karte einzeichnen könnten!

Es gibt mehrere Wege, wie man da hinkommt.

### a) mit Geopandas

Wenn wir in unserer Jupyter-Notebook-Umgebung bleiben wollen, können wir Geopandas nutzen.

Wie das genau funktioniert, besprechen wir später noch ausführlich. Hier aber schonmal ein Beispiel.

**Shapefile laden**

Um etwas auf einer Karte einzeichen zu können, brauchen wir... eine Karte!

In der GIS-Welt heissen solche Karten "Shapefiles". Wir haben eines für die Schweizer Gemeinden zur Verfügung:

In [ ]:
gdf = gpd.read_file('dataprojects/wahlen/shp/bfs-2019-wahl-gemeinden.shp')

Wir laden das Shapefile in ein so genanntes Geodataframe:

In [ ]:
gdf

In [ ]:
gdf.plot(color='lightblue', figsize=(15,9))

**Daten mit Shapefile verbinden**

Damit wir auf der Karte etwas einzeichnen können, müssen wir zu jeder Gemeinde eine Information hinzufügen.

Wir nehmen dazu unsere gewohnte Auswertung. Diesmal: Wie hoch ist der Wähleranteil der Grünen 2019?

In [ ]:
partei = 'GPS'

In [ ]:
df_partei = df[df['Partei_Name'] == partei]

In [ ]:
df_partei.head()

... und matchen sie mit `merge()` ins Geodatenframe:

In [ ]:
gdf_partei = gdf.merge(df_partei, how='inner', left_on='id', right_on='Gemeinde_ID')

In [ ]:
gdf_partei.head()

**Plotten**

Geopandas hat eine Plot-Funktion, die ähnlich funktioniert wie in Pandas.

Wichtig: wir müssen angeben, anhand von welcher Information die Farbgebung erstellt werden soll!

In [ ]:
# Plot
ax = gdf_partei.plot(column='2019', legend=True, figsize=(15,9))

ax.set_title("Wähleranteil 2019 in Prozentpunkten " + partei)

Standardmässig werden die Daten anhand einer kontinuierlichen Skala dargestellt.

Wir können diese Skala auch austauschen. Eine Auswahl von Skalen gibt es hier:

https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html

Wichtig auch: Die Min/Max-Werte einstellen.

In [ ]:
# Plot
ax = gdf_partei.plot(
    column='2019',
    cmap='Greens',
    vmin=0,
    vmax=30,
    figsize=(15,9),
    legend=True)

ax.set_title("Wähleranteil 2019 in Prozentpunkten " + partei)

Das Herumspielen mit Minimal- und Maximalwerten lohnt sich. Es kommt drauf an, was man hervorheben will!

Wir können uns auch eigene Grenzwerte erstellen:

In [ ]:
bins = [
    0, 5, 10, 15, 20, 25
]

In [ ]:
# Plot
ax = gdf_partei.plot(
    column='2019',
    cmap='Greens',
    scheme='user_defined',
    classification_kwds={'bins': bins},
    figsize=(15,9),
    legend=True)

ax.set_title("Wähleranteil 2019 in Prozentpunkten " + partei)

### b) mit einem externen Tool

zum Beispiel: Datawrapper. https://www.datawrapper.de/

Dort hat es diverse Maps bereits vorprogrammiert - auch die Schweizer Gemeinden.

Um es zu benutzen, müssen wir die Daten exportieren:

In [ ]:
party = 'GPS'

In [ ]:
year = '2019'

In [ ]:
df_temp = df[df['Partei_Name'] == party][['Gemeinde_ID', 'Gemeinde_Name', year]]
df_temp.to_csv('dataprojects/wahlen/charts/' + party + '-' + year + '.csv', index=False)